**bold text**
# LawLite ⚖️

**LawLite** is a fine-tuned Large Language Model (LLM) designed to simplify complex legal documents into clear, plain-English summaries. Built on top of open-source LLaMA-2-7B model and optimized using **LoRA/QLoRA** techniques, LawLite helps users quickly understand lengthy contracts, agreements, and compliance documents without legal jargon.

With a focus on **accuracy, clarity, and accessibility**, LawLite bridges the gap between legal professionals and everyday readers by turning pages of dense text into concise, actionable insights.

here i am going to use this dataset to fine tune the model: https://zenodo.org/records/7152317#.Yz6mJ9JByC0

This cell installs the required Python libraries for the project:

* peft → for parameter-efficient fine-tuning of large models

* accelerate → to easily manage multi-GPU / distributed training

* bitsandbytes → for memory-efficient 8-bit/4-bit model training

* transformers → Hugging Face’s library for working with LLMs

* datasets → for handling and processing datasets

In [ ]:
!pip install peft
!pip install accelerate
!pip install bitsandBytes
!pip install transformers
!pip install datasets

Installs GPUtil, a utility library that helps in checking the current GPU usage (memory, load, etc.), which is useful before training large models.

In [ ]:
!pip install GPUtil

here in this cell we are going to:

* Imports PyTorch, GPUtil, and os.

* Displays current GPU utilization with GPUtil.showUtilization().

* Checks if a GPU is available for training and prints the status.

* Configures CUDA environment variables to ensure GPU device ordering and explicitly sets the visible device to GPU 0.

In [ ]:
import torch
import GPUtil
import os

GPUtil.showUtilization()

if torch.cuda.is_available():
    print("GPU is available")
else:
    print("GPU is not available, using CPU instead")

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

Imports all the necessary Hugging Face and PEFT utilities.

* AutoTokenizer / AutoModelForCausalLM → load pre-trained LLMs

* BitsAndBytesConfig → configure quantization

* LlamaTokenizer → specific tokenizer for LLaMA models

* notebook_login → authenticate with Hugging Face Hub

* peft utilities for LoRA fine-tuning

In [ ]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, LlamaTokenizer
from huggingface_hub import notebook_login
from datasets import load_dataset
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

if "COLAB_GPU" in os.environ:
  from google.colab import output
  output.enable_custom_widget_manager()

Authenticates with Hugging Face Hub.

Uses CLI login if on Google Colab.

Otherwise, uses the notebook_login() widget.

In [ ]:
if "COLAB_GPU" in os.environ:
  !huggingface-cli login
else:
  notebook_login()

The below cell loads the LLaMA 2–7B chat model in 4-bit quantized format using bitsandbytes.
This drastically reduces memory usage, making training possible on limited GPUs.

In [ ]:
base_model_id = "meta-llama/Llama-2-7b-chat-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config)

we have already uploaded a dataset in zipflie format which consists of the supreme court judgements of Indian supreme court and UK supreme court which is already divided into train and test data

you can also download the data from here:https://zenodo.org/records/7152317#.Yz6mJ9JByC0

the below cell handles dataset upload and extraction:

* Assumes a .zip dataset is uploaded.

* Extracts it to /content/dataset/.

* Lists files to confirm extraction.

In [ ]:
from google.colab import files
import zipfile
import os

# Path to your uploaded zip
zip_path = "/content/dataset.zip"

# Unzip into /content/dataset/
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("/content/dataset")

# Check extracted files
os.listdir("/content/dataset")

Loads Indian legal judgment dataset into Hugging Face datasets.

* Reads .txt files from train-data and test-data folders.

* Creates train and test datasets.

* Prints dataset details.

In [ ]:
from datasets import load_dataset
import os

# Paths to train/test
base_path = "/content/dataset/dataset/IN-Abs"
train_path = os.path.join(base_path, "train-data/judgement")
test_path  = os.path.join(base_path, "test-data/judgement")

# Load dataset
train_dataset = load_dataset(
    "text",
    data_files={"train": [os.path.join(train_path, f) for f in os.listdir(train_path) if f.endswith(".txt")]},
    split="train"
)

test_dataset = load_dataset(
    "text",
    data_files={"test": [os.path.join(test_path, f) for f in os.listdir(test_path) if f.endswith(".txt")]},
    split="test"
)

print(train_dataset)
print(test_dataset)

The below cell performs foll action:

* Loads tokenizer for the LLaMA 2 model.

* Ensures there is a padding token (uses EOS token if missing).

In [ ]:
tokenizer = LlamaTokenizer.from_pretrained(base_model_id, use_fast=False, trust_remote_code=True, add_eos_token=True)

if tokenizer.pad_token is None:
  tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

The below cell tokenizes the training dataset:

* Converts each text sample into tokens.

* Applies truncation and padding up to 512 tokens.

In [ ]:
# Tokenize the train dataset from your uploaded IN data
tokenized_train_dataset = []
for phrase in train_dataset:
    tokenized = tokenizer(phrase["text"], truncation=True, padding="max_length", max_length=512)
    tokenized_train_dataset.append(tokenized)


Displays one tokenized example from the training dataset.

In [ ]:
tokenized_train_dataset[1]

Checks what the End-of-Sequence (EOS) token is for the tokenizer.

In [ ]:
tokenizer.eos_token

Prepares the model for LoRA fine-tuning:

* Enables gradient checkpointing (saves memory).

* Defines a LoRA config with rank=8, dropout=0.05.

* Applies LoRA adapters to attention and projection layers.

In [ ]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=8,
    lora_alpha=64,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)

The below cell sets up Hugging Face Trainer for fine-tuning:

* Uses LoRA-modified LLaMA model.

* Batch size = 2, with gradient accumulation = 2.

* Trains for 3 epochs (or max 20 steps).

* Uses paged AdamW optimizer with 8-bit efficiency.

* Saves checkpoints and logs.

* Starts training with trainer.train().

In [ ]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    args=transformers.TrainingArguments(
        output_dir="./finetunedModel",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=2,
        num_train_epochs=3,
        learning_rate=1e-4,
        max_steps=20,
        bf16=False,
        optim="paged_adamw_8bit",
        logging_dir="./log",
        save_strategy="epoch",
        save_steps=50,
        logging_steps=10

),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache=False
trainer.train()

Reloads the base LLaMA 2 model with 4-bit quantization for inference.

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig, LlamaTokenizer
from peft import PeftModel

base_model_id = "meta-llama/Llama-2-7b-chat-hf"

nf4Config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = LlamaTokenizer.from_pretrained(base_model_id, use_fast=False, trust_remote_code=True, add_eos_token=True)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=nf4Config,
    device_map="auto",
    trust_remote_code=True,
    use_auth_token=True
  )

Loads the fine-tuned LoRA model checkpoint on top of the base LLaMA 2 model.

In [ ]:
tokenizer = LlamaTokenizer.from_pretrained(base_model_id, use_fast=False, trust_remote_code=True, add_eos_token=True)

modelFinetuned = PeftModel.from_pretrained(base_model, "finetunedModel/checkpoint-20")

The below cell tests the fine-tuned model:

* Defines a legal question prompt.

* Tokenizes it and sends it to the model.

* Generates a response (up to 1024 tokens).

* Prints the model’s prediction.

In [ ]:
user_question = "Appeal from the High Court of judicature, Bombay, in a reference under section 66 of the Indian Income tax Act, 1022.K.M. Munshi , for the lant.  M.C. Setalvad, Attorney General for India, for the respondent. 1950."

eval_prompt = f"Question: {user_question} Just answer this question accurately and concisely.\n"

promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

modelFinetuned.eval()

with torch.no_grad():
  print(tokenizer.decode(modelFinetuned.generate(**promptTokenized, max_new_tokens=1024)[0], skip_special_tokens=True))
  torch.cuda.empty_cache()

Lets create a Gradio interface to see the working

In [ ]:
# Import/install Gradio
try:
    import gradio as gr
except:
    !pip -q install gradio
    import gradio as gr

print(f"Gradio version: {gr.__version__}")

In [ ]:
import gradio as gr

# Inference function
def chat_with_model(user_input):
    prompt = f"Question: {user_input}\nAnswer concisely:\n"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.1
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Gradio UI
demo = gr.Interface(
    fn=chat_with_model,
    inputs=gr.Textbox(lines=5, placeholder="Enter a legal question..."),
    outputs="text",
    title="⚖️ LawLite: Legal AI Assistant",
    description="Ask me Indian legal questions, and I will provide concise answers using a fine-tuned LLaMA model."
)

if __name__ == "__main__":
    demo.launch(debug=True,share=True)